In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
from xgboost import XGBClassifier
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


In [2]:
train_titanic=pd.read_csv("/kaggle/input/titanic/train.csv")
rows_original=train_titanic.shape[0]
col_original=train_titanic.shape[1]
train_titanic.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
#checking for females survival rate
females=female_survived=train_titanic[(train_titanic["Sex"]=="female")]
f=len(females)
female_survived=train_titanic[(train_titanic["Sex"]=="female") & (train_titanic["Survived"]==1 )]
fs=len(female_survived)
# This is out of the total number of the females
female_survived_percent=len(female_survived)/len(females)*100
print(f"No of females survived out of the {f} is {female_survived_percent:.2f}% females in the titanic where the total number of females who survived is {fs}.")
# This would be out of the total numbers of the passengers so
female_survived_percent_as_whole=fs/rows_original*100
print(f"Out of total {rows_original} passengers, total of {fs} survived and their percentage is {female_survived_percent_as_whole:.2f}%.")


No of females survived out of the 314 is 74.20% females in the titanic where the total number of females who survived is 233.
Out of total 891 passengers, total of 233 survived and their percentage is 26.15%.


In [4]:
#creating new feature to see total family aboard
train_titanic["total_family_aboard"]=train_titanic["SibSp"]+train_titanic["Parch"]
train_titanic.iloc[200:,:]

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,total_family_aboard
200,201,0,3,"Vande Walle, Mr. Nestor Cyriel",male,28.0,0,0,345770,9.5000,NaN,S,0
201,202,0,3,"Sage, Mr. Frederick",male,NaN,8,2,CA. 2343,69.5500,NaN,S,10
202,203,0,3,"Johanson, Mr. Jakob Alfred",male,34.0,0,0,3101264,6.4958,NaN,S,0
203,204,0,3,"Youseff, Mr. Gerious",male,45.5,0,0,2628,7.2250,NaN,C,0
204,205,1,3,"Cohen, Mr. Gurshon ""Gus""",male,18.0,0,0,A/5 3540,8.0500,NaN,S,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S,0
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S,0
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S,3
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C,0


In [5]:
#checking for honorifics
pattern=r"(\bMr\.|Mrs\.|Master\.|Miss\.|Dr\.|Rev\.|Col\.|Mile\.|Ms\.|Mrs\.|Jonkheer\.\b)"
train_titanic["honorifics"]=train_titanic["Name"].str.extract(pattern,expand=False).str.replace(".","",regex=False)
train_titanic["honorifics"].fillna("Others")
train_titanic.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,total_family_aboard,honorifics
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,1,Mr
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,1,Mrs
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,0,Miss
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,1,Mrs
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,0,Mr


In [6]:
#to check if they had cabin or not
train_titanic["has_cabin"]=train_titanic["Cabin"].notna().astype(int)
train_titanic.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,total_family_aboard,honorifics,has_cabin
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,1,Mr,0
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,1,Mrs,1
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,0,Miss,0
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,1,Mrs,1
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,0,Mr,0


In [7]:
# since we can see that ones in the upper class, females and children lets take some column and create array of numpy and feed it to the xgb classifier
features = ["Pclass", "Sex", "SibSp", "Parch","total_family_aboard","honorifics","has_cabin"]
output=train_titanic["Survived"]
inputs=train_titanic[features]
x=inputs.to_numpy()
y=output.to_numpy()

In [8]:
from sklearn.ensemble import RandomForestClassifier

y = train_titanic["Survived"]


X = pd.get_dummies(train_titanic[features])
test_data = pd.read_csv("/kaggle/input/titanic/test.csv")
test_data["total_family_aboard"]=test_data["SibSp"]+test_data["Parch"]
test_data["honorifics"]=test_data["Name"].str.extract(pattern,expand=False).str.replace(".","",regex=False)
test_data["honorifics"].fillna("Others")
test_data["has_cabin"]=test_data["Cabin"].notna().astype(int)

X_test = pd.get_dummies(test_data[features])

model = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=1)
model.fit(X, y)
predictions = model.predict(X_test)

output = pd.DataFrame({'PassengerId': test_data.PassengerId, 'Survived': predictions})
output.to_csv('submission.csv', index=False)
